<a href="https://www.kaggle.com/code/anzarwani2/pyspark-beginner-tutorial?scriptVersionId=196733461" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/air-quality/Air_Quality.csv
/kaggle/input/electric-power-consumption/powerconsumption.csv


# 1. Setting Up PySpark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812364 sha256=548d4586201ed49a0c92f60f0671a417a16a45d3bbae7985999a81de0887cb26
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('pyspark tutorial') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/15 10:17:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[1] appName=pyspark tutorial>
Spark App Name : pyspark tutorial


In [5]:
data = [("Alice", 25, "New York"),
        ("Raghu", 30, "Bengaluru"),
        ("Ali", 29, "Pune")]

# Define the schema for the DataFrame
columns = ["Name", "Age", "City"]

# Create the DataFrame
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

+-----+---+---------+
| Name|Age|     City|
+-----+---+---------+
|Alice| 25| New York|
|Raghu| 30|Bengaluru|
|  Ali| 29|     Pune|
+-----+---+---------+



# 2. DataFrame Operations

In [6]:
df_csv = spark.read.csv("/kaggle/input/electric-power-consumption/powerconsumption.csv", header=True, inferSchema=True)

# Show the DataFrame
df_csv.show(5)

+-------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|     Datetime|Temperature|Humidity|WindSpeed|GeneralDiffuseFlows|DiffuseFlows|PowerConsumption_Zone1|PowerConsumption_Zone2|PowerConsumption_Zone3|
+-------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|1/1/2017 0:00|      6.559|    73.8|    0.083|              0.051|       0.119|            34055.6962|           16128.87538|           20240.96386|
|1/1/2017 0:10|      6.414|    74.5|    0.083|               0.07|       0.085|           29814.68354|           19375.07599|           20131.08434|
|1/1/2017 0:20|      6.313|    74.5|     0.08|              0.062|         0.1|           29128.10127|           19006.68693|           19668.43373|
|1/1/2017 0:30|      6.121|    75.0|    0.083|              0.091|       0.096|           28228.86076|    

In [7]:
#filtering

df_wind = df_csv.filter(df_csv.WindSpeed > 2.0)
df_wind.show(10)

+--------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|      Datetime|Temperature|Humidity|WindSpeed|GeneralDiffuseFlows|DiffuseFlows|PowerConsumption_Zone1|PowerConsumption_Zone2|PowerConsumption_Zone3|
+--------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|1/1/2017 10:10|      5.836|    71.3|     2.66|              257.9|       31.01|               25920.0|           15837.08207|           14428.91566|
|1/1/2017 10:20|      5.996|   69.85|     4.93|              282.7|       31.96|           26393.92405|           16059.57447|           14671.80723|
|1/1/2017 10:30|       6.22|   68.81|    4.924|              307.0|       32.42|           26861.77215|           16322.18845|           15036.14458|
|1/1/2017 10:40|      6.703|   68.01|    4.923|              327.6|       33.22|           27511.898

In [8]:
#multiple filters

df_wind_temp = df_csv.filter((df_csv.WindSpeed > 2.0) & ((df_csv.Temperature >10)))
df_wind_temp.show(10)

+--------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|      Datetime|Temperature|Humidity|WindSpeed|GeneralDiffuseFlows|DiffuseFlows|PowerConsumption_Zone1|PowerConsumption_Zone2|PowerConsumption_Zone3|
+--------------+-----------+--------+---------+-------------------+------------+----------------------+----------------------+----------------------+
|1/6/2017 13:10|      15.22|   65.98|    2.287|              348.7|       308.3|           32409.11392|           21680.24316|           16811.56627|
|1/6/2017 13:20|      15.38|   66.11|    4.708|              244.2|       234.6|           33022.78481|           21902.73556|           16724.81928|
|1/20/2017 6:00|      10.09|   62.24|    2.981|              0.095|       0.082|           24157.97468|           15709.42249|           14839.51807|
|1/20/2017 6:10|      10.73|   56.13|    4.917|              0.066|       0.115|           24394.936

In [9]:
df_air = spark.read.csv("/kaggle/input/air-quality/Air_Quality.csv", header = True, inferSchema = True)

df_air.show(10)

+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+-------+
|Unique ID|Indicator ID|                Name|       Measure|Measure Info|Geo Type Name|Geo Join ID|      Geo Place Name|        Time Period|Start_Date|Data Value|Message|
+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+-------+
|   179772|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        409|    Southeast Queens|               2015|01/01/2015|       0.3|   NULL|
|   179785|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        209|Bensonhurst - Bay...|               2015|01/01/2015|       1.2|   NULL|
|   178540|         365|Fine particles (P...|          Mean|      mcg/m3|        UHF42|        209|Bensonhurst - Bay...|Annual Average 2012|12/01

In [10]:
# group by

from pyspark.sql import functions as F

grouped_df = df_air.groupBy("Geo Place Name").agg(
    F.avg("Data Value").alias("Average_Data_value")
)

# Show the result
grouped_df.show()

+--------------------+------------------+
|      Geo Place Name|Average_Data_value|
+--------------------+------------------+
|     Northeast Bronx| 22.30780669144981|
|Belmont and East ...|20.756363636363634|
|Upper East Side (...| 25.25181818181818|
|Greenwich Village...| 21.53529411764706|
|              Queens|  19.3964705882353|
|  East Harlem (CD11)| 23.88090909090909|
|     Lower Manhattan|23.359999999999996|
|         Northern SI|15.122222222222225|
|Central Harlem - ...| 33.37063197026022|
|Throgs Neck and C...|18.146363636363635|
|South Crown Heigh...|19.159999999999997|
|Stapleton - St. G...|20.343529411764713|
|Bayside Little Ne...|15.822222222222228|
|Fort Greene and B...|22.967272727272725|
|       Midtown (CD5)|26.824545454545458|
|Riverdale and Fie...|16.596363636363638|
|Union Square - Lo...|29.379999999999985|
|Williamsbridge an...| 17.22090909090909|
|Highbridge and Co...|21.458181818181828|
|  Washington Heights|24.133828996282514|
+--------------------+------------

# 3. Data Manipulation

In [11]:
#rename a column

df_air.withColumnRenamed('Geo Place Name', 'Place').show(5) #it returns a new dataframe

+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+-------+
|Unique ID|Indicator ID|                Name|       Measure|Measure Info|Geo Type Name|Geo Join ID|               Place|        Time Period|Start_Date|Data Value|Message|
+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+-------+
|   179772|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        409|    Southeast Queens|               2015|01/01/2015|       0.3|   NULL|
|   179785|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        209|Bensonhurst - Bay...|               2015|01/01/2015|       1.2|   NULL|
|   178540|         365|Fine particles (P...|          Mean|      mcg/m3|        UHF42|        209|Bensonhurst - Bay...|Annual Average 2012|12/01

In [12]:
#select specific columns

df_air.select("Name", "Data Value").show(5)

+--------------------+----------+
|                Name|Data Value|
+--------------------+----------+
|Boiler Emissions-...|       0.3|
|Boiler Emissions-...|       1.2|
|Fine particles (P...|       8.6|
|Fine particles (P...|       8.0|
|Fine particles (P...|       6.1|
+--------------------+----------+
only showing top 5 rows



In [13]:
#drop a column

df_air.drop('Message').show(5)

+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+
|Unique ID|Indicator ID|                Name|       Measure|Measure Info|Geo Type Name|Geo Join ID|      Geo Place Name|        Time Period|Start_Date|Data Value|
+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-------------------+----------+----------+
|   179772|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        409|    Southeast Queens|               2015|01/01/2015|       0.3|
|   179785|         640|Boiler Emissions-...|Number per km2|      number|        UHF42|        209|Bensonhurst - Bay...|               2015|01/01/2015|       1.2|
|   178540|         365|Fine particles (P...|          Mean|      mcg/m3|        UHF42|        209|Bensonhurst - Bay...|Annual Average 2012|12/01/2011|       8.6|
|   178561|         36

## Adding Columns

1. A constant value
2. Based on a condition

In [14]:
data = [("Alice", 25, "New York", 25000),
        ("Raghu", 30, "Bengaluru", 45000),
        ("Ali", 29, "Pune", 36000),
       ("Dominic", 20, "London", 25000),
       ("Sara", 19, "Tunis", 12000)]

columns = ["Name", "Age", "City", "Salary"]

df = spark.createDataFrame(data, columns)

df.show()

+-------+---+---------+------+
|   Name|Age|     City|Salary|
+-------+---+---------+------+
|  Alice| 25| New York| 25000|
|  Raghu| 30|Bengaluru| 45000|
|    Ali| 29|     Pune| 36000|
|Dominic| 20|   London| 25000|
|   Sara| 19|    Tunis| 12000|
+-------+---+---------+------+



In [15]:
#Add column with constant value
from pyspark.sql.functions import lit, when, mean

df.withColumn('Favorite Music Genre', lit("Pop")).show(5)

+-------+---+---------+------+--------------------+
|   Name|Age|     City|Salary|Favorite Music Genre|
+-------+---+---------+------+--------------------+
|  Alice| 25| New York| 25000|                 Pop|
|  Raghu| 30|Bengaluru| 45000|                 Pop|
|    Ali| 29|     Pune| 36000|                 Pop|
|Dominic| 20|   London| 25000|                 Pop|
|   Sara| 19|    Tunis| 12000|                 Pop|
+-------+---+---------+------+--------------------+



In [16]:
# Add column based on condition

df.withColumn('Generation', when((df.Age > 12) & (df.Age < 27), "Gen Z")
             .when((df.Age > 28) & (df.Age < 43), "Millennials")).show(5)

+-------+---+---------+------+-----------+
|   Name|Age|     City|Salary| Generation|
+-------+---+---------+------+-----------+
|  Alice| 25| New York| 25000|      Gen Z|
|  Raghu| 30|Bengaluru| 45000|Millennials|
|    Ali| 29|     Pune| 36000|Millennials|
|Dominic| 20|   London| 25000|      Gen Z|
|   Sara| 19|    Tunis| 12000|      Gen Z|
+-------+---+---------+------+-----------+



# 4. Handling Missing Data

In [17]:
data = [("Alice", 25, "New York", 25000),
        ("Raghu", None, "Bengaluru", 45000),
        ("Ali", 29, None, 36000),
        (None, 20, "London", 25000),
        ("Sara", 19, "Tunis", None)]

columns = ["Name", "Age", "City", "Salary"]

df = spark.createDataFrame(data, columns)

df.show()

+-----+----+---------+------+
| Name| Age|     City|Salary|
+-----+----+---------+------+
|Alice|  25| New York| 25000|
|Raghu|NULL|Bengaluru| 45000|
|  Ali|  29|     NULL| 36000|
| NULL|  20|   London| 25000|
| Sara|  19|    Tunis|  NULL|
+-----+----+---------+------+



In [18]:
# dropping rows with missing values

df_dropped = df.dropna()
df_dropped.show()

+-----+---+--------+------+
| Name|Age|    City|Salary|
+-----+---+--------+------+
|Alice| 25|New York| 25000|
+-----+---+--------+------+



In [19]:
# filling missing values with specific values

mean_salary = df.select(mean(df['Salary'])).collect()[0][0]

df_filled = df.fillna({
    "Name": "Unknown",  # Fill missing names with "Unknown"
    "Age": 0,           # Fill missing ages with 0
    "City": "Unknown",   # Fill missing cities with "Unknown"
    "Salary": mean_salary         # Fill missing salaries with mean salary
})

df_filled.show()

+-------+---+---------+------+
|   Name|Age|     City|Salary|
+-------+---+---------+------+
|  Alice| 25| New York| 25000|
|  Raghu|  0|Bengaluru| 45000|
|    Ali| 29|  Unknown| 36000|
|Unknown| 20|   London| 25000|
|   Sara| 19|    Tunis| 32750|
+-------+---+---------+------+



# 5. Joins and Aggregations

In [20]:
data1 = [(1, 'Chocolate', 200.00, 'Food'),
        (2, 'Cookies', 100.00, 'Food'),
        (3, 'Journal', 250.00, 'Stationary'),
        (4, 'Energy Drink', 300.00, 'Beverage'),
        (5, 'Noodles', 230.00, 'Food')]

data1Cols = ['item_id', 'item_name', 'item_price', 'item_category']

store1 = spark.createDataFrame(data1, data1Cols)

store1.show()

+-------+------------+----------+-------------+
|item_id|   item_name|item_price|item_category|
+-------+------------+----------+-------------+
|      1|   Chocolate|     200.0|         Food|
|      2|     Cookies|     100.0|         Food|
|      3|     Journal|     250.0|   Stationary|
|      4|Energy Drink|     300.0|     Beverage|
|      5|     Noodles|     230.0|         Food|
+-------+------------+----------+-------------+



In [21]:
data2 = [(1, 5),
        (2, 3),
        (3, 2),
        (4, 10),
        (5, 7),
        (6,5),
        (7,10)]

data2Cols = ['item_id', 'item_quantity']

store2 = spark.createDataFrame(data2, data2Cols)

store2.show()

+-------+-------------+
|item_id|item_quantity|
+-------+-------------+
|      1|            5|
|      2|            3|
|      3|            2|
|      4|           10|
|      5|            7|
|      6|            5|
|      7|           10|
+-------+-------------+



In [22]:
# inner join

store1.join(store2, store1.item_id == store2.item_id, "inner").show()

+-------+------------+----------+-------------+-------+-------------+
|item_id|   item_name|item_price|item_category|item_id|item_quantity|
+-------+------------+----------+-------------+-------+-------------+
|      1|   Chocolate|     200.0|         Food|      1|            5|
|      2|     Cookies|     100.0|         Food|      2|            3|
|      3|     Journal|     250.0|   Stationary|      3|            2|
|      4|Energy Drink|     300.0|     Beverage|      4|           10|
|      5|     Noodles|     230.0|         Food|      5|            7|
+-------+------------+----------+-------------+-------+-------------+



In [23]:
# left join

store1.join(store2, store1.item_id == store2.item_id, "left").show()

+-------+------------+----------+-------------+-------+-------------+
|item_id|   item_name|item_price|item_category|item_id|item_quantity|
+-------+------------+----------+-------------+-------+-------------+
|      5|     Noodles|     230.0|         Food|      5|            7|
|      1|   Chocolate|     200.0|         Food|      1|            5|
|      3|     Journal|     250.0|   Stationary|      3|            2|
|      2|     Cookies|     100.0|         Food|      2|            3|
|      4|Energy Drink|     300.0|     Beverage|      4|           10|
+-------+------------+----------+-------------+-------+-------------+



In [24]:
# right join

store1.join(store2, store1.item_id == store2.item_id, "right").show()

+-------+------------+----------+-------------+-------+-------------+
|item_id|   item_name|item_price|item_category|item_id|item_quantity|
+-------+------------+----------+-------------+-------+-------------+
|   NULL|        NULL|      NULL|         NULL|      7|           10|
|   NULL|        NULL|      NULL|         NULL|      6|            5|
|      5|     Noodles|     230.0|         Food|      5|            7|
|      1|   Chocolate|     200.0|         Food|      1|            5|
|      3|     Journal|     250.0|   Stationary|      3|            2|
|      2|     Cookies|     100.0|         Food|      2|            3|
|      4|Energy Drink|     300.0|     Beverage|      4|           10|
+-------+------------+----------+-------------+-------+-------------+



In [25]:
join_df = store1.join(store2, on='item_id')
join_df.withColumn('total_revenue', join_df.item_price * join_df.item_quantity).show()

+-------+------------+----------+-------------+-------------+-------------+
|item_id|   item_name|item_price|item_category|item_quantity|total_revenue|
+-------+------------+----------+-------------+-------------+-------------+
|      1|   Chocolate|     200.0|         Food|            5|       1000.0|
|      2|     Cookies|     100.0|         Food|            3|        300.0|
|      3|     Journal|     250.0|   Stationary|            2|        500.0|
|      4|Energy Drink|     300.0|     Beverage|           10|       3000.0|
|      5|     Noodles|     230.0|         Food|            7|       1610.0|
+-------+------------+----------+-------------+-------------+-------------+



# 6. Working with RDDs (Basic)

**Will load a text file and count the number of occurences of a word
in this text file**

In [26]:
textFile = '''
This is a sample text file to learn basic map and
reduce function, map and reduce will be used
to calculate the number of occurences of a word
in this text file
'''

with open('mpText.txt', 'w') as f:
    f.write(textFile)

In [27]:
rdd = spark.sparkContext.textFile('/kaggle/working/mpText.txt')

In [28]:
# Split each line into words
words_rdd = rdd.flatMap(lambda line: line.split())

# Map each word to a tuple (word, 1) and reduce by key to count occurrences
word_count_rdd = words_rdd.map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)

In [29]:
word_count_rdd.saveAsTextFile("word_count_output")

In [30]:
for word, count in word_count_rdd.collect():
    print(f"{word}: {count}")

this: 2
is: 1
a: 2
sample: 1
text: 2
file: 2
to: 2
learn: 1
basic: 1
map: 2
and: 2
reduce: 2
function,: 1
will: 1
be: 1
used: 1
calculate: 1
the: 1
number: 1
of: 2
occurences: 1
word: 1
in: 1
